In [1]:
# MathUtil.ipynb 파일 내에 정의된 메서드를 호출합니다. 
# TODO : 경로를 확인하여 주세요.
%run ./MathUtil.ipynb

In [ ]:
# Dataset 클래스의 역할 
# 목적 : 다양한 데이터셋을 지원하기 위한 데이터 클래스(부모)
# 목적 : 다양한 데이터셋 마다 자식 클래스로 관리
class Dataset(object):
    print("Dataset 클래스 호출")

    # __init__ 메서드 
    # 목적 : AI 에 적용된 데이터셋과 AI Mode 를 확인하기 위함 
    # dataset_name : AI 를 위한 학습 데이터 셋의 이름을 입력합니다. 
    # ai_mode : AI Mode 설정 (regression, binary, select, dual_select 중 활용되는 데이터에 맞게 표기)
    def __init__(self, dataset_name, ai_mode):
        self.dataset_name = dataset_name 
        self.ai_mode = ai_mode 
    
    
    # __str__ 메서드 
    # 목적 : 데이터의 세부 정보를 확인하기 위해 출력을 조정합니다. 
    def __str__(self):
        return 'Dataset : {} | AI Mode : {} | Train : {} | Test : {} | Val : {}'.format(self.dataset_name, self.ai_mode, len(self.tr_xs), len(self.te_xs), len(self.va_xs))

    # train_count() 메서드 
    # 목적 : 학습 데이터 수를 반환
    # 데코레이터 특성 상 메서드이나 메서드가 아닌 속성으로 취급. 
    # 'a.train_count()' 형식 대신, 인수 구조 없이 a.train_count 형식으로 호출 가능
    @property
    def train_count(self):
        return len(self.tr_xs)

In [3]:
# Dataset.shuffle_train_data 의 역할
# 목적 : 학습 데이터의 인덱스 생성 및 셔플링
# size : 배치 사이즈와 1 에폭에 따른 배치 수 기반, 학습 데이터 수
def dataset_shuffle_train_data(self, size):
    print("Dataset.shuffle_train_data 호출")
    
    self.indices = np.arange(size)  # self.indices 변수 생성
    np.random.shuffle(self.indices) # 인덱싱 셔플링 
    # 위와 같이 생성된 self.indices 는 추후 데이터를 나누는 역할에 사용됩니다. 

# Dataset.get_train_data의 역할 
# batch_size : 기본값 10, 사용자가 입력한 batch_size 에 맞게 시작과 끝 인덱싱 생성 
# nth : 1 Epoch 에 따른 배치 인덱싱 
def dataset_get_train_data(self, batch_size, nth):
    print("Dataset.get_train_data 호출")
    
    # batch_size 에 기반한 시작과 끝 인덱스 생성 
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size

    # office_image_init() -> self.shuffle_data() 메서드의 동작때 생성되는 
    # self.tr_xs, self.tr_ys 데이터를 인덱싱 정보를 활용하여 
    # 학습 데이터의 독립변수, 종속변수를 생성 및 반환 합니다.
    tr_X = self.tr_xs[self.indices[from_idx:to_idx]]
    tr_Y = self.tr_ys[self.indices[from_idx:to_idx]]

    return tr_X, tr_Y

Dataset.shuffle_train_data = dataset_shuffle_train_data
Dataset.get_train_data = dataset_get_train_data

In [4]:
# Dataset.get_test_data 의 역할
# 목적 : 메서드 동작 시 사전에 확보된 te_xs, te_ys 값을 반환.
def dataset_get_test_data(self):
    print("Dataset.get_test_data 호출") 
    return self.te_xs, self.te_ys

Dataset.get_test_data = dataset_get_test_data

In [5]:
# 본 메서드는 같은 기능으로 두 메서드로 정의합니다.
# Dataset.get_validate_data 의 역할
# 목적 : 검증 데이터를 반환.
# count : 모델 테스트 당시 희망하는 검증 데이터의 수

# Dataset.get_visualize_data 의 역할 
# 목적 : 일부 결과에 대한 시각화를 위해 필요한 검증 데이터 확보
# count : 시각화를 위해 일부 데이터를 호출하는 과정에서 필요한 데이터의 수 
def dataset_get_validate_data(self, count):
    print("dataset_get_validate_data() 호출")
    
    # 사전 확보된 검증 데이터의 인덱스를 확보합니다. 
    self.va_indices = np.arange(len(self.va_xs))
    # 인덱스의 셔플링 
    np.random.shuffle(self.va_indices)

    # 검증 데이터를 독립변수와 종속 변수로 분리하여 반환합니다.
    va_X = self.va_xs[self.va_indices[0:count]]
    va_Y = self.va_ys[self.va_indices[0:count]]

    return va_X, va_Y

Dataset.get_validate_data = dataset_get_validate_data
Dataset.get_visualize_data = dataset_get_validate_data

In [ ]:
# Dataset.shuffle_data 의 역할 
# 목적 : 
# 1. 전체 독립변수와 종속 변수를 전달받습니다. 이후 독립변수와 종속변수에 대한 학습, 검증, 테스트 데이터를 생성합니다. 
# 2. 독립 변수와 종속변수에 대한 차원 수를 변수화 합니다. 
# xs : 독립변수
# ys : 종속변수 
# tr_ratio : 학습 데이터의 비율 
# va_ratio : 검증 데이터의 비율
def dataset_shuffle_data(self, xs, ys, tr_ratio=0.8, va_ratio=0.05):
    print("Dataset.shuffle_data 호출")
    
    # 독립 변수 전체 데이터 수를 확인합니다. 
    data_count = len(xs)
    
    # 학습 데이터는 가까운 10의 배수로 내리는 것을 목표합니다. 
    # 독립변수의 수(data_count) 와 학습 데이터의 비율을 곱한 이후 10으로 나눠 이를 int()메서드로 처리하면 정숫값을 반환받습니다. 이러한 상황에서 다시 10을 곱하여 가까운 10의 배수값을 얻습니다. 
    tr_cnt = int(data_count * tr_ratio / 10) * 10

    # 검증 데이터의 수는 독립변수의 수와 검증 데이터의 비율을 곱하여 값을 확인합니다. 
    va_cnt = int(data_count * va_ratio)
    # 테스트 데이터는 독립변수의 수 에서 학습 데이터와 검증 데이터를 더한 값의 편차를 구하는 식으로 연산을 처리합니다. 
    te_cnt = data_count - (tr_cnt + va_cnt)

    # 학습 데이터, 검증 데이터, 테스트 데이터의 시작, 끝 인덱스 확보
    tr_from, tr_to = 0, tr_cnt
    va_from, va_to = tr_cnt, tr_cnt + va_cnt
    te_from, te_to = tr_cnt + va_cnt, data_count

    # 전체 독립변수의 인덱스를 확보하고 셔플링을 진행합니다. 
    indices = np.arange(data_count)
    np.random.shuffle(indices)

    # 전달 받은 독립변수(xs)와 종속변수(ys)를 인덱스를 활용하여 
    # 학습, 검증, 테스트 데이터에 대한 독립변수와 종속변수를 얻습니다. 
    self.tr_xs = xs[indices[tr_from:tr_to]]
    self.tr_ys = ys[indices[tr_from:tr_to]]
    self.va_xs = xs[indices[va_from:va_to]]
    self.va_ys = ys[indices[va_from:va_to]]
    self.te_xs = xs[indices[te_from:te_to]]
    self.te_ys = ys[indices[te_from:te_to]]

    # 첫 번째 독립변수의 첫 번째 차원값을 input_shape 변수를 생성합니다. 
    # MlpModel.init_parameters 메서드 내에 파라미터를 생성하는 과정에서 
    # self.dataset.input_shape 변수로 활용
    
    # 첫 번째 종속변수의 첫 번째 차원값을 output_shape 변수를 생성합니다. 
    # MlpModel.init_parameters 메서드 내에 파라미터를 생성하는 과정에서 
    # self.dataset.output_shape 변수로 활용
    # Image 데이터의 경우 
    # xs[0].shape는 [100,100,3] 을 감안하여 30000 이라는 값을 갖습니다. 
    # ys[0].shape는 종속변수의 개수에 맞게 값이 설정되며, 34개의 종속변수를 갖는경우 34라는 값을 갖습니다. 
    self.input_shape = xs[0].shape
    self.output_shape = ys[0].shape
    
Dataset.shuffle_data = dataset_shuffle_data

In [7]:
# Dataset.forward_postproc 의 역할
# 목적 : ai_mode 에 맞춰서 loss 값 반환, 역전파에 필요한 변수 반환 
# output : 예측값
# y : 실제 값 
# ai_mode : 순전파에 활용되는 ai_mode 
def dataset_forward_postproc(self, output, y, ai_mode=None):
    print("Dataset.forward_postproc 호출")
    
    # 만약 ai_mode 값이 기본 값(None)이라면 
    # 사전에 정의된 self.ai_mode 값을 ai_mode 변수로 정의
    if ai_mode is None : 
        ai_mode = self.ai_mode
    
    # 만약 'regression' 이라면 MSE 
    if ai_mode == 'regression':
        
        # 회귀 순전파의 손실 / MSE
        diff = output - y
        square = np.square(diff)
        loss = np.mean(square)
        
        # MSE의 역전파에 필요한 보조지표
        aux = diff
    
    # 만약 'binary' 이라면 시그모이드 크로스 엔트로피
    elif ai_mode == 'binary':
        
        # 손실 연산
        # MathUtil.ipynb 파일에 정의된 메서드입니다. 별도의 클래스 처리가 되어 있지 않음.
        entropy = sigmoid_cross_entropy_with_logits(y, output)
        # 손실에 대한 평균을 loss 로 정의 
        loss = np.mean(entropy)

        # 시그모이드 교차 엔트로피의 역전파에 필요한 보조지표
        aux = [y, output]
    
    # 만약 'select' 이라면 소프트맥스 교차 엔트로피
    elif ai_mode == 'select':

        # 손실 연산
        # MathUtil.ipynb 파일에 정의된 메서드입니다. 별도의 클래스 처리가 되어 있지 않음.
        entropy = softmax_cross_entropy_with_logits(y, output)
        # 손실에 대한 평균을 loss 로 정의
        loss = np.mean(entropy)

        # 소프트맥스 교차 엔프로피의 역전파에 필요한 보조지표
        aux = [output, y, entropy]

    # 보조지표 반환    
    return loss, aux

Dataset.forward_postproc = dataset_forward_postproc

In [8]:
# Dataset.backprop_postproc 의 역할
# 목적 : ai_mode 에 맞춰서 G_output 반환
# 세부 설명 : 
# ∂L/∂W 혹은 ∂L/∂B 를 구하기 위해 필요한 ∂L/∂Y 편미분 연산 (변수명 : G_output)
# 가중치 업데이트 수식 일부 ∂L/∂W = ∂L/∂Y * ∂Y/∂W
# 편향 업데이트 수식 일부 ∂L/∂B = ∂L/∂Y * ∂Y/∂B

# G_loss : 기울기의 초깃값 G_loss = 1.0
# aux : 각 처리 방식에 맞는 G_output 연산 
# ai_mode : 각 처리 방식에 맞는 ai_mode 매개변수 확보
def dataset_backprop_postproc(self, G_loss, aux, ai_mode=None):
    print("Dataset.backprop_postproc 호출")

    # ai_mode 값이 기본 값(None)이라면, 매개변수로 넘어온 값을 그대로 활용
    if ai_mode is None: 
        ai_mode = self.ai_mode
    
    # 매개변수로 넘어온 값이 'regression' 이라면 회귀에 맞는 G_output 변수 연산
    if ai_mode == 'regression':
        diff = aux # aux = diff
        shape = diff.shape
        
        g_loss_square = np.ones(shape) / np.prod(shape)
        g_square_diff = 2 * diff
        g_diff_output = 1

        G_square = g_loss_square * G_loss
        G_diff = g_square_diff * G_square
        G_output = g_diff_output * G_diff

    # 매개변수로 넘어온 값이 'binary' 이라면 그에 맞는 G_output 변수 연산
    elif ai_mode == 'binary':
        y, output = aux # aux = [y, output]
        shape = output.shape

        g_loss_entropy = np.ones(shape) / np.prod(shape)
        g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    
    # 매개변수로 넘어온 값이 'select' 이라면 그에 맞는 G_output 변수 연산
    elif ai_mode == 'select':
        output, y, entropy = aux # aux = [output, y, entropy]

        g_loss_entropy = 1.0 / np.prod(entropy.shape)
        g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    
    return G_output

Dataset.backprop_postproc = dataset_backprop_postproc

In [9]:
# Dataset.eval_accuracy 의 역할
# 목적 : ai_mode 에 맞춰서 정확도 연산
# 다음과 같은 코드를 통해 호출 됩니다. 
# acc0 = self.base.eval_accuracy(x, ys[0], outputs[0], 'select')
# acc1 = self.base.eval_accuracy(x, ys[1], outputs[1], 'select')

# x : 독립변수 
# y : 종속변수 
# output : 예측
# ai_mode : AI 모드 
def dataset_eval_accuracy(self, x, y, output, ai_mode=None):
    print("Dataset.eval_accuracy 호출")
    
    # ai_mode 에 맞는 정확도 연산 
    # ※ 주의 : binary 와 select 의 경우 매우 단순한 일치 여부 중심
    if ai_mode is None :
        ai_mode = self.ai_mode
    
    if ai_mode == 'regression':
        mse = np.mean(np.square(output - y))
        accuracy = 1 - np.sqrt(mse) / np.mean(y)

    elif ai_mode == 'binary':
        estimate = np.greater(output, 0)
        answer = np.equal(y, 1.0)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
        
    elif ai_mode == 'select':
        estimate = np.argmax(output, axis=1)
        answer = np.argmax(y, axis=1)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
        
    return accuracy

Dataset.eval_accuracy = dataset_eval_accuracy

In [10]:

# Dataset.get_estimate 의 역할
# 목적 : ai_mode 에 맞춰서 output 을 추측변수(estimate)로 할당

def dataset_get_estimate(self, output, ai_mode=None):
    print("Dataset.get_estimate 호출")
    if ai_mode is None: 
        ai_mode = self.ai_mode
    
    # 회귀의 경우 output 과 estimate 는 별도의 처리가 없습니다. 
    if ai_mode == 'regression':
        estimate = output

    # 이진판단의 경우 예측값(output)에 대하여 sigmoid() 메서드 할당 
    elif ai_mode == 'binary':
        estimate = sigmoid(output)

    # 이진판단의 경우 예측값(output)에 대하여 softmax() 메서드 할당 
    elif ai_mode == 'select':
        estimate = softmax(output)
    
    # 예측값 반환 
    return estimate

Dataset.get_estimate = dataset_get_estimate

In [ ]:
# Dataset.train_prt_result 의 역할
# Dataset.test_prt_result
# 목적 : 일반적인 데이터인 경우 동작하는 학습 및 테스트 결과 출력 메서드 
def dataset_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    print("Dataset.train_prt_result 호출")
    print('[Epoch {}] cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)'. \
          format(epoch, np.mean(costs), np.mean(accs), acc, time1, time2))

def dataset_test_prt_result(self, name, acc, time):
    print("Dataset.test_prt_result 호출")
    print(' {} AI Model Test report: accuracy = {:5.3f}, ({} secs)\n'. \
          format(name, acc, time))

Dataset.train_prt_result = dataset_train_prt_result
Dataset.test_prt_result = dataset_test_prt_result